In [57]:
## TRABAJO FINAL - DETECTOR DE PERSONAS vFinal
## Detector automatico de personas en una imagen. El programa genera un modelo a traves de imagenes de entrenamiento y
## y luego busca personas en una imagen cualquiera.
## version: trabajo_final_v14
## Autores
    ## Ayesa Moneo, Ander
    ## Fernandez Ortega, Unai Javier
    ## Soba Jimenez, Iñigo
    
# Librerias
import os
import cv2
import numpy as np
from skimage.feature import hog
from skimage import exposure
from sklearn.datasets import make_classification
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pandas as pd
from sklearn import svm
from skimage.transform import pyramid_gaussian

import time

In [58]:
# Genera el histograma de una imagen y lo ecualiza
# param im: imagen que queremos ecualizar
# return: imagen ecualizada
def equalizar_imagen(im):
    
    hist,bins = np.histogram(im.flatten(), 256, [0,256])
    cdf = hist.cumsum()
    cdf_normalized = cdf * hist.max() / cdf.max()

    cdf_m = np.ma.masked_equal(cdf,0)
    cdf_m = (cdf_m - cdf_m.min()) * 255 / (cdf_m.max( )- cdf_m.min())
    cdf = np.ma.filled(cdf_m, 0).astype('uint8')

    imagenFinal= cdf[im]
    return imagenFinal

In [59]:
# Convierte a gris las imagenes contenidas en la direccion recibida
# param tipo: tipo de las imagenes que queremos convertir a gris (train / test)
# param clase: clase de las imagenes que queremos convertir a gris (pos / neg)
def convertir_a_gris(tipo,clase):
    
    lista = os.listdir(os.getcwd() + '//' + tipo + '/' + clase)
    
    for i in lista:
        im = cv2.imread(os.getcwd() + '//' + tipo + '/' + clase + '/' + i, 0)    
        im = equalizar_imagen(im)
        cv2.imwrite(os.getcwd() + '//' + tipo + '/' + clase + '_gris/' + i, im)


In [60]:
# Duplica las imagenes contenidas en la ruta 
# param tipo: tipo de las imagenes que queremos convertir a gris (train / test)
# param clase: clase de las imagenes que queremos convertir a gris (pos / neg)
def duplicar_personas(tipo,clase):
    
    lista=os.listdir(os.getcwd() + '//' + tipo + '/' + clase)
    num=len(lista)
    
    for i in lista:
        im=cv2.imread(os.getcwd() + '//' + tipo + '/' + clase + '/' + i, 0)
        aux=np.fliplr(im)
        cv2.imwrite(os.getcwd() + '//' + tipo + '/' + clase + '/' + str(num) + '.jpg', aux)
        num += 1

In [61]:
# Crea subimagenes a partir de las imagenes de clase negativa
# param tipo: tipo de las imagenes que queremos convertir a gris (train / test)
# param clase: clase de las imagenes que queremos convertir a gris (pos / neg)
# param num_subimg: numero de subimagenes que queremos por cada imagen
# param width: tamaño de anchura que queremos que tengan las subimagenes obtenidas
# param height: tamaño de altura que queremos que tengan las subimagenes obtenidas
def subimagenes_neg(tipo,clase,num_subimg,width,height):
    
    lista=os.listdir(os.getcwd() + '//' + tipo + '/' + clase)
    num = 1
    
    for i in lista:
        im = cv2.imread(os.getcwd() + '//' + tipo + '/' + clase + '/' + i, 0)
        for j in range(num_subimg):
            eje_x = np.random.choice(im.shape[0] - height, 1)[0]
            eje_y = np.random.choice(im.shape[1] - width, 1)[0]
            aux = im[eje_x:eje_x + height, eje_y:eje_y + width]
            cv2.imwrite(os.getcwd() + '//' + tipo + '/subimg_neg_gris/' + str(num) + '.jpg', aux)
            num += 1
             

In [62]:
# HOG (multichannel False por grises, pixels_per_cell)
# Crea vectores de caracteristicas de las imagenes en un directorio y los devuelve como una matriz de caracteristicas
# param tipo: tipo de las imagenes que queremos convertir a gris (train / test)
# param destino: directorio de destino de las imagenes
# param clase: clase de las imagenes que queremos convertir a gris (pos / neg)
# return: matriz de caracteristicas del conjunto de imagenes
def HOG(clase, destino, tipo):
    
    lista = os.listdir(os.getcwd() + '//' + tipo + '/' + clase)
    matriz_caracteristicas = np.zeros((0, 0))   
    for i in lista:
        image = cv2.imread(os.getcwd() + '//' + tipo + '/' + clase + '/' + i, 0)    
        fd = hog(image, orientations = 8, pixels_per_cell = (32, 32),
        cells_per_block = (1, 1))
        fd = fd.reshape(1, -1)
        if matriz_caracteristicas.size == 0:
            matriz_caracteristicas = fd
        else:
            matriz_caracteristicas = np.vstack((matriz_caracteristicas,fd))

    return matriz_caracteristicas

In [63]:
# Prepara el entorno para realizar el entrenamiento de los modelos
# param mc_pos: matriz de caracteristicas de las imagenes positivas
# param mc_neg: matriz de caracteristicas de las imagenes negativas
# return: matriz de caracteristicas conjunta y matriz de validacion con filas aleatorias de la matriz de caracteristicas
def preparar_entrenamiento(mc_pos, mc_neg):
    
    #Preparamos train como la concatenación de train_pos y train_neg (Positiva 0 persona y negativa 1 no persona)
    mc_pos_con_Clase = np.hstack((mc_pos, np.zeros((mc_pos.shape[0], 1))))
    mc_neg_con_Clase = np.hstack((mc_neg, np.ones((mc_neg.shape[0], 1))))

    matriz_caracteristicas = np.vstack((mc_pos_con_Clase, mc_neg_con_Clase))
    np.random.shuffle(matriz_caracteristicas)

    #Vamos a extraer una cantidad de filas para validacion
    N = 150
    aleat = np.random.randint(0, matriz_caracteristicas.shape[0] - N)

    matriz_validacion = matriz_caracteristicas[aleat:aleat + N,:]
    matriz_caracteristicas = np.delete(matriz_caracteristicas, slice(aleat, aleat + N), 0)
    
    return matriz_caracteristicas, matriz_validacion

In [64]:
# Implementa un algoritmo de regresion logistica
# param matriz_caracteristicas: matriz de caracteristicas usada para el entrenamiento
# param matriz_validacion: matriz de caracteristicas de las imagenes usadas para validacion
def modelo_regresion_logistica(matriz_caracteristicas, matriz_validacion):
    #Importamos el modelo de regresion logistica
    lr = LogisticRegression()
    #Ahora vamos a entrenar el modelo
    lr.fit(matriz_caracteristicas[:, :-1], matriz_caracteristicas[:, -1])
        #Imprimimos ciertas caracteristicas del entrenamiento
#     print(lr.coef_)
#     print(lr.intercept_)
        #Ahora vamos a probar la precision con el conjunto de validacion
    y_pred = lr.predict(matriz_validacion[:,:-1])

        #Vamos a ver la precision
    solucion = matriz_validacion[:, -1] == y_pred
    print('Accuracy train lr: ',solucion[solucion == True].shape[0] / solucion.shape[0])
    
    return lr

In [65]:
#Implementa un algoritmo de support vector machine(svm)
# param matriz_caracteristicas: matriz de caracteristicas usada para el entrenamiento
# param matriz_validacion: matriz de caracteristicas de las imagenes usadas para validacion
def modelo_svm(matriz_caracteristicas, matriz_validacion):
    
    svc = svm.LinearSVC(C = 10.0, loss = 'hinge', max_iter = 10000)
    svc.fit(matriz_caracteristicas[:, :-1], matriz_caracteristicas[:, -1])
    y_pred = svc.predict(matriz_validacion[:, :-1])

    #Vamos a ver la precision
    solucion = matriz_validacion[:, -1] == y_pred
    print('Accuracy train svm: ',solucion[solucion == True].shape[0] / solucion.shape[0])
    
    return svc

In [66]:
#Redimensiona un conjunto de imagenes al tamaño de la ventana
# param tipo: tipo de las imagenes que queremos convertir a gris (train / test)
# param clase: clase de las imagenes que queremos convertir a gris (pos / neg)
def convertir_tamano(tipo, clase):
    
    lista = os.listdir(os.getcwd() + '//' + tipo + '/' + clase)
    for i in lista:
        im = cv2.imread(os.getcwd() + '//' + tipo + '/' + clase + '/' + i, 0)
        imResize = cv2.resize(im, (70, 134))
        cv2.imwrite(os.getcwd() + '//' + tipo + '/' + clase + '_tamano/' + i, imResize)

In [67]:
# Realiza una prediccion en base al modelo generado con regresion logistica
# param matriz_caracteristicas_test_pos: matriz de caracteristicas de las imagenes positivas de test
# param matriz_caracteristicas_test_neg: matriz de caracteristicas de las imagenes negativas de test
def prediccion_lr(matriz_caracteristicas_test_pos, matriz_caracteristicas_test_neg,lr):
    
    y_pred = lr.predict(matriz_caracteristicas_test_pos[:,:])
    y_pred2 = lr.predict(matriz_caracteristicas_test_neg[:,:])

    print('Accuracy test positivo lr: ',y_pred[y_pred == 0].size / y_pred.shape[0])
    print('Accuracy test negativo lr: ',y_pred2[y_pred2 == 1].size / y_pred2.shape[0])

In [68]:
# Realiza una prediccion en base al modelo generado con support vector machine
# param matriz_caracteristicas_test_pos: matriz de caracteristicas de las imagenes positivas de test
# param matriz_caracteristicas_test_neg: matriz de caracteristicas de las imagenes negativas de test
def prediccion_svm(matriz_caracteristicas_test_pos, matriz_caracteristicas_test_neg,svc):
    
    y_pred = svc.predict(matriz_caracteristicas_test_pos[:,:])
    y_pred2 = svc.predict(matriz_caracteristicas_test_neg[:,:])

    print('Accuracy test positivo svm: ',y_pred[y_pred == 0].size / y_pred.shape[0])
    print('Accuracy test negativo svm: ',y_pred2[y_pred2 == 1].size / y_pred2.shape[0])

In [69]:
# Recorre la imagen usando una ventana deslizante y crea un vector que contenga esas ventanas
# param im: imagen que queremos recorrer
# param modelo: modelo entrenado
# param tam_ventana: tamaño de la ventana deslizante
# param salto: cantidad de pixeles que avanza la ventana cada vez
# return: vector de ventanas recorridas
def ventana_deslizante(im, modelo, tam_ventana, salto):
    
    vector_ventanas = []
    for i in range(0, im.shape[0], salto[1]):
        for j in range(0, im.shape[1], salto[0]):
            ventana = im[i:i + tam_ventana[1],j:j + tam_ventana[0]]
            vector_ventanas.append((i, j, ventana))
   
    return vector_ventanas

In [70]:
#Analiza la imagen en base al modelo. Si hay una persona la rodea con un rectangulo y muestra la imagen
# param im: imagen que queremos analizar
# param modelo: modelo entrenado para reconocer personas
# param tam_ventana: tamaño de la ventana con la que recorremos la imagen
# param salto: cantidad de pixeles que avanzamos la ventana deslizante cada vez
def analisis(im,modelo,tam_ventana,salto):
    start_time = time.time()
    imAux = np.copy(im)
    img_aux = im.copy()
    detectados = []
    escala = 0
    downscale = 1.5
    for imagen_escalada in pyramid_gaussian(im, downscale = downscale):
        if(imagen_escalada.shape[0] < 134 or imagen_escalada.shape[1] < 70):
            break
        for(i, j, ventana) in ventana_deslizante(imagen_escalada, modelo, tam_ventana, salto):
            if(ventana.shape[0] != 134 or ventana.shape[1] != 70):
                continue
            fd = hog(ventana, orientations = 8, pixels_per_cell = (32, 32), cells_per_block=(1, 1),
                     visualize = False)
            fd = fd.reshape(1, -1)
            pred = modelo.predict(fd)
            
            if(pred == 0):   
                if(np.abs(modelo.decision_function(fd))>1.5):
                    detectados.append((int(j * (downscale**escala)), int(i * (downscale**escala)),
                              modelo.decision_function(fd),
                              int(70 * (downscale**escala)),
                              int(134 * (downscale**escala))))
            else:
                continue#print("NO HAY PERSONA")  
        escala += 1
        
    for(x, y, c, d, e) in detectados:
        cv2.rectangle(im, (x, y), (x + d, y + e), (0,0,255), thickness = 2)

    rectangulos = np.array([[i, j, i + d, j + e] for(i, j, c, d, e) in detectados])
    rectangulos_finales = non_max_suppression(rectangulos, 0)
    for(x1, y1, x2, y2) in rectangulos_finales:
        cv2.rectangle(img_aux, (x1, y1), (x2, y2), (0,0,255), 2)
    print("Tiempo de ejecucion analisis--- %s seconds ---" % (time.time() - start_time))        
    cv2.imshow("PERSONA sin supresion de no maximos",im)
    cv2.waitKey(0)
    cv2.imshow("PERSONA 2 con supresion de no maximos",img_aux)
    cv2.waitKey(0)
    cv2.destroyAllWindows()          


In [71]:
def non_max_suppression(boxes, overlapThresh):

    # Convertir enteros de los bounding boxes a float
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")
 
    pick = []
 
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
 
    # compute the area of the bounding boxes and sort the bounding
    # boxes by the bottom-right y-coordinate of the bounding box
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(x1)

    # keep looping while some indexes still remain in the indexes
    # list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the
        # index value to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        # find the largest (x, y) coordinates for the start of
        # the bounding box and the smallest (x, y) coordinates
        # for the end of the bounding box
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        # compute the width and height of the bounding box
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        # compute the ratio of overlap
        overlap = (w * h) / area[idxs[:last]]
        # delete all indexes from the index list that have
        idxs = np.delete(idxs, np.concatenate(([last],
            np.where(overlap > overlapThresh)[0])))

    # return only the bounding boxes that were picked using the
    # integer data type
    return boxes[pick].astype("int")

PROGRAMA PRINCIPAL

In [83]:
start_time = time.time()

# Convertimos a gris las imagenes que usamos en el entrenamiento
convertir_a_gris('train', 'train_pos')
convertir_a_gris('train', 'train_neg')
print("Tiempo de conversion a gris total--- %s seconds ---" % (time.time() - start_time))
# Duplicamos las imagenes positivas para tener mas cantidad
duplicar_personas('train', 'train_pos_gris')

# Obtenemos las subimagenes de las imagenes negativas para tener mas imagenes con las que entrenar
subimagenes_neg('train', 'train_neg_gris', 3, 70, 134)

# Obtenemos las matrices de caracteristicas de las imagenes positivas y negativas
matriz_caracteristicas_pos=HOG('train_pos_gris','hog_train_pos','train')
matriz_caracteristicas_neg=HOG('subimg_neg_gris','hog_train_neg', 'train')

#Pasamos a blanco y negro las imagenes de test
convertir_a_gris('test', 'neg')
convertir_a_gris('test', 'pos')

#Ahora creamos la carpeta con las imagenes redimensionadas
convertir_tamano('test','neg_gris')
convertir_tamano('test','pos_gris')

#Ahora vamos a tomar las imagenes de test y vamos a predecir en test        
matriz_caracteristicas_test_pos = HOG('pos_gris_tamano', 'hog_train_pos', 'test')
matriz_caracteristicas_test_neg = HOG('neg_gris_tamano', 'hog_train_neg', 'test')



matriz_caracteristicas,matriz_validacion=preparar_entrenamiento(matriz_caracteristicas_pos, matriz_caracteristicas_neg)
#Entrenamos los modelos
lr=modelo_regresion_logistica(matriz_caracteristicas, matriz_validacion)
svc=modelo_svm(matriz_caracteristicas, matriz_validacion)

#Hacemos la prediccion con ambos modelos
prediccion_lr(matriz_caracteristicas_test_pos, matriz_caracteristicas_test_neg,lr)
prediccion_svm(matriz_caracteristicas_test_pos, matriz_caracteristicas_test_neg,svc)

#Ejecutamos el programa principal con una imagen de prueba
im=cv2.imread('prueba_4.PNG',0)
#Se llama a la funcion principal del código
vector_ventanas = analisis(im, lr,[70,134],[10,20])

print("Tiempo de ejecucion total--- %s seconds ---" % (time.time() - start_time))

Tiempo de conversion a gris total--- 3.9062440395355225 seconds ---


C:\ProgramData\Anaconda3\envs\py37vision\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy train lr:  0.84


C:\ProgramData\Anaconda3\envs\py37vision\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Accuracy train svm:  0.8666666666666667
Accuracy test positivo lr:  0.96
Accuracy test negativo lr:  0.7
Accuracy test positivo svm:  0.9
Accuracy test negativo svm:  0.9


C:\ProgramData\Anaconda3\envs\py37vision\lib\site-packages\skimage\transform\_warps.py:23: UserWarning: The default multichannel argument (None) is deprecated.  Please specify either True or False explicitly.  multichannel will default to False starting with release 0.16.
  warn('The default multichannel argument (None) is deprecated.  Please '
C:\ProgramData\Anaconda3\envs\py37vision\lib\site-packages\skimage\transform\_warps.py:23: UserWarning: The default multichannel argument (None) is deprecated.  Please specify either True or False explicitly.  multichannel will default to False starting with release 0.16.
  warn('The default multichannel argument (None) is deprecated.  Please '


Tiempo de ejecucion analisis--- 1.0156142711639404 seconds ---
Tiempo de ejecucion total--- 36.783427715301514 seconds ---


In [ ]:
#Para no tener que volver a ejecutar el procesamiento de todas la imagenes, puedes ejecutar esta casilla 
# cambiando el nombre de la imagen
im=cv2.imread('prueba_3.jpg',0)
#Se llama a la funcion principal del código
vector_ventanas = analisis(im, svc,[70,134],[10,20])